In [1]:
import pandas as pd
import os
import glob
import numpy as np
import statistics as stats
import scipy
import src.mpra_tools.predicted_occupancy as po
import src.mpra_tools.fasta_utils as fu
import math
import matplotlib.pyplot as plt
from collections import Counter, defaultdict
import random
from src.mpra_tools.fasta_utils import write_fasta

In [3]:
activity_df = pd.read_csv("Data/activity.csv", index_col=0)
retinopathy_df = pd.read_csv("Data/retinopathy.csv", index_col=0)
L = 164

### Create fasta file for retinopathy data

In [4]:
# ret_s = pd.Series(
#     data = retinopathy_df['sequence'],
#     index=retinopathy_df.index
# )
# write_fasta(ret_s, "Data/Fasta/retinopathy.fasta")

In [8]:
fimo_df = pd.read_csv("Data/Motifs/fimo_eLife_retinopathy/fimo.tsv", sep='\t')
del fimo_df['motif_alt_id']
fimo_df.dropna(inplace=True)
fimo_df['motif_id'] = fimo_df['motif_id'].map(lambda x: x.split('_')[0])
fimo_df.head()

,motif_id,sequence_name,start,stop,strand,score,p-value,q-value,matched_sequence
0,MEF2D,CRX.4455,121.0,134.0,-,19.4478,9.400000e-09,0.00489,TGCTATTTTTAGCC
1,MEF2D,CRX.1423,110.0,123.0,-,19.0896,4.370000e-08,0.01140,AGCTATTTTTAGCT
2,MAZ,CRX.269,3.0,13.0,+,18.6970,1.130000e-07,0.00737,GAGGGAGGGGG
3,MAZ,CRX.3724,3.0,13.0,-,18.6970,1.130000e-07,0.00737,GAGGGAGGGGG
4,MAZ,CRX.4201,41.0,51.0,+,18.6970,1.130000e-07,0.00737,GAGGGAGGGGG


In [9]:
po_df = pd.read_parquet("Data/Motifs/retinopathy_motifs.parquet")
po_df.head()

motif strand  length  occupancy
label    position                                
CRX.1003 11        NDF1      R      10   0.209839
         42         CRX      F       8   0.354500
         54         CRX      R       8   0.291680
         64         CRX      R       8   0.641077
         110       NDF1      F      10   0.804350

In [10]:
test = activity_df[activity_df['activity_bin']=='potato']
test.empty

True